In [ ]:
!pip install openpyxl
!pip install awswrangler
import openpyxl
import awswrangler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import boto3
import io
import os
# s3_path = "s3://<your-bucket-name>/food-twentieth-century-crop-statistics-1900-2017-xlsx.xlsx"
s3_path = "s3://c93435a2086654l4910637t1w0854223549-sandboxbucket-yk2arc2gss8l/food-twentieth-century-crop-statistics-1900-2017-xlsx.xlsx"
df = awswrangler.s3.read_excel(s3_path, sheet_name="CropStats", usecols=lambda name: name != 'Unnamed: 0')
print(df.head())

In [ ]:
df.drop(['admin2', 'notes', 'Harvest_year'], axis=1, inplace=True)
df.rename(columns = {'admin0': 'national', 'admin1': 'subnational', 'hectares (ha)': 'hectares_ha', 'production (tonnes)': 'production_tonnes', 'yield(tonnes/ha)': 'yield_tonnes_ha'}, inplace=True)
df.loc[df['subnational'].isna(), 'subnational'] = df['national']
# Calculate yield
mask = df['yield_tonnes_ha'].isna() & ~df['production_tonnes'].isna() & ~df['hectares_ha'].isna() & df['hectares_ha'] != 0
df.loc[mask, 'yield_tonnes_ha'] = df['production_tonnes'] / df['hectares_ha']
df.dropna(subset=['yield_tonnes_ha'], inplace=True)
# Calculate production
mask = df['production_tonnes'].isna() & ~df['yield_tonnes_ha'].isna() & ~df['hectares_ha'].isna()
df.loc[mask, 'production_tonnes'] = df['yield_tonnes_ha'] * df['hectares_ha']
df.dropna(subset=['production_tonnes'], inplace=True)
# Calculate hectares
mask = df['hectares_ha'].isna() & ~df['yield_tonnes_ha'].isna() & ~df['production_tonnes'].isna()
df.loc[mask, 'hectares_ha'] = df['yield_tonnes_ha'] * df['production_tonnes']
df.dropna(subset=['hectares_ha'], inplace=True)
# The columns we just adapted just changed into objects, let's make them floats again
df['hectares_ha'] = df['hectares_ha'].astype(float)
df['production_tonnes'] = df['production_tonnes'].astype(float)
df['yield_tonnes_ha'] = df['yield_tonnes_ha'].astype(float)
# Filter for wheat
df = df[df['crop'] == 'wheat']
# Remove the crop column
df.drop('crop', axis=1, inplace =True)
# Logarithmic transformations
df['log_yield'] = np.log1p(df['yield_tonnes_ha'])
df['log_hectares'] = np.log1p(df['production_tonnes'])
df['log_production'] = np.log1p(df['hectares_ha'])
print(df.head())

In [ ]:
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
train, test_and_validate = train_test_split(df, 
                                            test_size=0.2, 
                                            random_state=42)
#                                             stratify=df_wine['quality'])
test, validate = train_test_split(test_and_validate, 
                                  test_size=0.5, 
                                  random_state=42)
#                                   stratify=test_and_validate['quality'])
print(train.shape)
print(test.shape)
print(validate.shape)

In [ ]:
# bucket='c93435a2086654l4910637t1w0854223549-sandboxbucket-jtqmwpnickrd'
bucket='c93435a2086654l4910637t1w0854223549-sandboxbucket-yk2arc2gss8l'
prefix='CropDataModelHannah'
train_file='crop_train.csv'
test_file='crop_test.csv'
validate_file='crop_validate.csv'
whole_file='crop.csv'
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create a LinearRegression object
lr = LinearRegression()

# Fit the model to the training data
lr.fit(train[['log_hectares', 'log_production']], train['log_yield'])

# Make predictions on the test data
test_predictions = lr.predict(test[['log_hectares', 'log_production']])

# Calculate the mean squared error (MSE) on the test data
mse = np.mean((test_predictions - test['log_yield'])**2)

# Print the MSE
print('MSE:', mse)

# Calculate the accuracy of the predictions
accuracy = np.mean(test_predictions == test['log_yield'])

# Print the accuracy
print('Accuracy:', accuracy)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create the scatter plot
plt.scatter(test_predictions, test_predictions - test['yield_tonnes_ha'])

# Add a title and axis labels
plt.title('Residual Analysis')
plt.xlabel('Predicted Yield (tonnes/ha)')
plt.ylabel('Residual (tonnes/ha)')

# Show the plot
plt.show()

In [ ]:
import boto3

s3 = boto3.client('s3')

In [ ]:
import botocore

s3_client = boto3.client('s3')

try:
    s3_client.get_bucket_acl(Bucket=bucket)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'AccessDenied':
        print('The s3 variable does not have permission to access the S3 bucket.')
        exit(1)

In [ ]:
import pickle

# Save the trained model to S3
with open('model.pkl', 'wb') as f:
    pickle.dump(lr, f)

# Upload the trained model to S3
s3.upload_file('model.pkl', bucket, os.path.join(prefix, 'model', 'model.pkl'))

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Make predictions on the test data
test_predictions = lr.predict(test[['log_hectares', 'log_production']])

# Flatten the test_predictions array
test_predictions = test_predictions.ravel()

# Calculate the FPR and TPR of the model
fpr, tpr, thresholds = roc_curve(test['log_yield'], test_predictions)

# Calculate the AUC of the model
auc = auc(fpr, tpr)

# Plot the ROC curve
fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.set_title('ROC Curve')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
plt.show()
